In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import Slope_Tracer
import os
import shutil
import h5py

In [ ]:
import imp
imp.reload(Slope_Tracer)

In [ ]:
rundir = '/home/z3500785/dedalus_rundir/'
shutil.rmtree(rundir+'ifields/');shutil.rmtree(rundir+'snapshots/');shutil.rmtree(rundir+'moments/');os.remove(rundir+'runparams.npz')
input_dict = dict(Slope_Tracer.default_input_dict)
lday = 1.0e5

input_dict['dt'] = 2.*lday
#input_dict['Ttot'] = 320
input_dict['nz'] = 96
input_dict['AH'] = 100.
input_dict['AHvar'] = 0. # ONLY use 0 here if trying true along-isopycnal
input_dict['AHfull'] = 1

Slope_Tracer.run_sim(rundir,plot=True,**input_dict)

In [ ]:
Slope_Tracer.merge_move(rundir,'')

In [ ]:
# Plot ifields:
f, ax = plt.subplots(figsize=(10,5))
f.set_facecolor('white')
ax.set_xlabel('z');ax.set_ylabel('value')
with h5py.File(rundir + 'ifields/ifields.h5', mode='r') as file:
        y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
        ym, zm = np.meshgrid(y,z)
        zt = np.cos(theta)*zm + np.sin(theta)*ym
        yt = -np.sin(theta)*zm + np.cos(theta)*ym
        plt.plot(z,file['tasks']['KHyy'][0,0,:],label='KHyy')
        plt.plot(z,file['tasks']['KHyz'][0,0,:],label='KHyz')
        plt.plot(z,file['tasks']['KHzz'][0,0,:],label='KHzz')
#        plt.plot(z,file['tasks']['AHdd'][0,0,:],label='AHdd')
plt.xlim([0.,10.])
plt.legend()

In [ ]:
fig = plt.figure(figsize=(6, 5), facecolor='w')
with h5py.File(rundir + 'moments/moments.h5', mode='r') as file:
    t = file['scales/sim_time'][:]/lday;
    trT = file['tasks']['trT'][:,0,0]
    zm1i = file['tasks']['zm1i'][:,0,0]
    ym1i = file['tasks']['ym1i'][:,0,0]
    #plt.plot(t,trT/trT[0])
    plt.plot(t,zm1i/trT,'+k')


In [ ]:
outbase = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/';
outfold = outbase + 'prodruns_layer30-08-18/'
outdir = outfold + 'AH_010_ADV_2_Kinf_m5_mny0_0p5333_slope_200/';
rundir = '/home/z3500785/dedalus_rundir/';
# print(outdir)
# Slope_Tracer.merge_move(rundir,outdir)
os.makedirs(outdir, exist_ok=True)
shutil.move(rundir + 'snapshots/snapshots.h5',outdir + 'snapshots.h5');
shutil.move(rundir + 'moments/moments.h5',outdir + 'moments.h5');
shutil.move(rundir + 'ifields/ifields.h5',outdir + 'ifields.h5');
shutil.move(rundir + 'runparams.npz',outdir + 'runparams.npz');
shutil.rmtree(rundir + 'snapshots/');
shutil.rmtree(rundir + 'ifields/');
shutil.rmtree(rundir + 'moments/');


In [ ]:
slope = 1/100.0
theta = np.arctan(slope)
q0 = 1./30.
costh = np.cos(theta)
sinth = np.sin(theta)

# All coefficients of the equations:
z = np.linspace(0,200.,2000)

f = np.exp(-q0*z)*(np.cos(q0*z)+np.sin(q0*z));
fz = -2*q0*np.exp(-q0*z)*np.sin(q0*z);
GB2 = 1.+costh**2.*f*(f-2.)

dyC = fz*costh*sinth*(1+costh**2.*(-2.*f*f+3.*f-2))/(GB2**2.)
dzC = -fz*sinth*sinth*costh*costh*(3.*f-2.)/(GB2**2.)
dydyC = costh*costh*(1-f)**2./GB2
dydzC = -2*costh*sinth*(1-f)/GB2
dzdzC = sinth*sinth/GB2

plt.plot(z,(1.-f),label='1-f')
plt.plot(z,dyC/np.max(abs(dyC)),label='$\partial_y C$ coef.')
plt.plot(z,dzC/np.max(abs(dzC)),label='$\partial_z C$ coef.')
plt.plot(z,dydyC/np.max(abs(dydyC)),label='$\partial_y^2 C$ coef.')
plt.plot(z,dydzC/np.max(abs(dydzC)),label='$\partial_y\partial_z C$ coef.')
plt.plot(z,dzdzC/np.max(abs(dzdzC)),label='$\partial_z^2 C$ coef.')
plt.xlim([0.,60.])
plt.legend()

In [ ]:
z = np.linspace(0,200.,100)
q0 = 1./30.

f = np.exp(-q0*z)*(np.cos(q0*z)+np.sin(q0*z));
fz = -2*q0*np.exp(-q0*z)*np.sin(q0*z);
plt.plot(z,f)
plt.plot(z,fz)
plt.ylim([-0.1,0.1])
#Bz = 1. - np.exp(-q0*z)*(np.cos(q0*z)+np.sin(q0*z))
#AH = 1. - np.exp(-q0*z)
#tst2 = AH*f
#tst3 = np.exp(-q0*z)*(q0*f-2*q0*np.exp(-q0*z)*np.sin(q0*z) - q0)/q0
# plt.plot(Bz,z)
# plt.plot(AH,z,'-b')
# plt.plot(tst,z,'-r')
# plt.plot(tst2,z,'-m')
# plt.plot(tst3,z,'-g')

In [ ]:
import numpy as np
import h5py
import matplotlib
import shutil
import time
import matplotlib.pyplot as plt
from mpi4py import MPI
from scipy.special import erf
import os
from IPython import display
from dedalus import public as de
from dedalus.extras import flow_tools
from dedalus.tools import post
import pathlib
from dedalus.extras import plot_tools
import obsfit1d
import logging
logger = logging.getLogger(__name__)


In [ ]:
# Create bases and domain
z_basis = de.Chebyshev('z', 32, interval=(0, 1.))#, dealias=3/2)
domain = de.Domain([z_basis], grid_dtype=np.float64)
z1 = domain.grid(0)
z2 = domain.grid(0,scales=2.)
K = domain.new_field()
K['g'] = np.exp(-z1/0.2)


In [ ]:
f = plt.figure(figsize=(10,10),facecolor='w')
K.set_scales(1.)
plt.plot(K['g'],z1,'ok')
plt.plot(K['g']*0,z1,'ok')
K.set_scales(2.)
plt.plot(K['g'],z2,'om')
plt.plot(K['g']*0,z2,'om')

